# What is the best place to open a moderately priced, fast casual asian dessert spot in NYC?
### Considerations
* Where are all bakeries primarily aggregated?
* Using review count as a measure of success, does their price point affect the amount of reviews they recieve?
* Does the number/type of categories these businesses put on their profile affect their review_count?
* Integrate census data, how does median income affect these parameters?
    * Business 'success' using review_count (see if I can find out information for how long they've been open, take the average rev/year)
    * How income, price, success and location relate to eachother
  

In [19]:
# Import dependencies
import pandas as pd
import glob
import warnings

warnings.filterwarnings('ignore')

In [20]:
# Importing dataframes
dataframes = []
for file in glob.glob('Resources/*.csv'):
    dataframes.append(pd.read_csv(file))

# Concatenate dataframes
df_main = pd.concat(dataframes, ignore_index=True)
df_main.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,Unnamed: 0,category,coord_lat,coord_long
0,epvSBt9LldIgyc08f8nIAw,la-bicyclette-bakery-brooklyn-4,La Bicyclette Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/aPBPM3...,False,https://www.yelp.com/biz/la-bicyclette-bakery-...,3,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.5,"{'latitude': 40.68358704921419, 'longitude': -...",[],"{'address1': '305 Court St', 'address2': '', '...",NaN,NaN,2431.404071,NaN,NaN,NaN,NaN,NaN
1,-k_5NsYnKCHGTdBaqegkrw,le-fournil-new-york,Le Fournil,https://s3-media2.fl.yelpcdn.com/bphoto/JxNmFc...,False,https://www.yelp.com/biz/le-fournil-new-york?a...,118,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.5,"{'latitude': 40.727947, 'longitude': -73.988489}",['delivery'],"{'address1': '115 2nd Ave', 'address2': '', 'a...",NaN,NaN,2548.470477,$$,NaN,NaN,NaN,NaN
2,0kW0112jMERVjpRzWT4F7Q,la-bicyclette-bakery-brooklyn-3,La Bicyclette Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/_qCjVd...,False,https://www.yelp.com/biz/la-bicyclette-bakery-...,84,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",5.0,"{'latitude': 40.714400045936806, 'longitude': ...",['delivery'],"{'address1': '667 Driggs Ave', 'address2': Non...",1.347916e+10,(347) 916-1417,3128.240171,NaN,NaN,NaN,NaN,NaN
3,XSwagBljEsxG2i1rq4GEAQ,almondine-bakery-brooklyn,Almondine Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/n_j7ya...,False,https://www.yelp.com/biz/almondine-bakery-broo...,404,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.0,"{'latitude': 40.7033174, 'longitude': -73.9912...",['delivery'],"{'address1': '85 Water St', 'address2': None, ...",1.718798e+10,(718) 797-5026,348.916339,$$,NaN,NaN,NaN,NaN
4,fPQ2eE9lm8tc87O5-GLjyA,mille-feuille-bakery-new-york-3,Mille-Feuille Bakery,https://s3-media1.fl.yelpcdn.com/bphoto/HHm3mm...,False,https://www.yelp.com/biz/mille-feuille-bakery-...,441,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 40.782425, 'longitude': -73.9811603}","['delivery', 'pickup']","{'address1': '2175 Broadway', 'address2': '', ...",1.212363e+10,(212) 362-6261,8630.996628,$$,NaN,NaN,NaN,NaN


In [21]:
# Create sublist
master_df_sub = df_main[['id', 'name', 'review_count', 'categories', 'rating', 'coordinates', 'phone', 'display_phone', 'price']]
master_df_sub.head()

# View null values
print(master_df_sub.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11470 entries, 0 to 11469
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             11470 non-null  object 
 1   name           11470 non-null  object 
 2   review_count   11470 non-null  int64  
 3   categories     5098 non-null   object 
 4   rating         11470 non-null  float64
 5   coordinates    5098 non-null   object 
 6   phone          10215 non-null  float64
 7   display_phone  10215 non-null  object 
 8   price          7869 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 806.6+ KB
None


In [22]:
# Remove unnecessary labels
chars_to_remove = ['[', '{', '\'', 'alias', 'title', 'latitude', 'longitude', "}", ":", "}", "]"]
col_list = ['categories', 'coordinates']


def string_cleaner(df, col_list, char_list):
    for item in char_list:
        for col in col_list:
            df.loc[:, col] = df[col].str.replace(item, "", case=False)
    return df

string_cleaner(master_df_sub, col_list, chars_to_remove)

,id,name,review_count,categories,rating,coordinates,phone,display_phone,price
0,epvSBt9LldIgyc08f8nIAw,La Bicyclette Bakery,3,"bakeries, Bakeries",4.5,"40.68358704921419, -73.99509019999999",NaN,NaN,NaN
1,-k_5NsYnKCHGTdBaqegkrw,Le Fournil,118,"bakeries, Bakeries",4.5,"40.727947, -73.988489",NaN,NaN,$$
2,0kW0112jMERVjpRzWT4F7Q,La Bicyclette Bakery,84,"bakeries, Bakeries",5.0,"40.714400045936806, -73.95911000669003",1.347916e+10,(347) 916-1417,NaN
3,XSwagBljEsxG2i1rq4GEAQ,Almondine Bakery,404,"bakeries, Bakeries",4.0,"40.7033174, -73.9912506",1.718798e+10,(718) 797-5026,$$
4,fPQ2eE9lm8tc87O5-GLjyA,Mille-Feuille Bakery,441,"coffee, Coffee & Tea, bakeries, Bakeries",4.5,"40.782425, -73.9811603",1.212363e+10,(212) 362-6261,$$
...,...,...,...,...,...,...,...,...,...
11465,Ng_LY9KteuAMxggboYmVng,D'Orsi's Bakery,55,"catering, Caterers, bakeries, Bakeries",4.0,"40.56669, -74.2519499",1.732635e+10,(732) 634-7994,$$
11466,dt5ze7syLkALVI3DQfHp_A,Loqma Cafe,20,"turkish, Turkish, cafes, Cafes, seafood, ...",3.5,"40.585861, -73.953604",NaN,NaN,$$
11467,raqwXUBlnc0e8c7CYmxEJQ,Scarpetta,2264,"italian, Italian, bars, Bars, desserts, ...",4.0,"40.744509, -73.9856",1.212691e+10,(212) 691-0555,$$$
11468,1-k63Nhn0lRe48Zrz-WuJA,Miss American Pie,112,"bakeries, Bakeries, desserts, Desserts, c...",5.0,"40.68049, -73.97785",1.718789e+10,(718) 789-3747,NaN


In [23]:
# Tidying categories column
    # lower case all 
master_df_sub['categories'] = master_df_sub['categories'].astype('str').apply(lambda x: x.lower())
    # Split string value into list
master_df_sub['categories'] = master_df_sub['categories'].str.split(',')
    # Explode
master_df_sub = master_df_sub.explode('categories', ignore_index=True)
    # Remove hidden spacing
master_df_sub['categories'] = master_df_sub['categories'].str.strip()
    # Delete spacing for redundancy
master_df_sub['categories'] = master_df_sub['categories'].str.replace(' ', '')
    # Drop duplicates (alias), keep primary
master_df_sub.drop_duplicates(subset=['id', 'categories'], inplace=True, keep='first')
    # Aliases occasionally have underscores, filter these out
master_df_sub = master_df_sub[~master_df_sub['categories'].str.contains('_')]

# Preview
print(master_df_sub.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10413 entries, 0 to 29816
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10413 non-null  object 
 1   name           10413 non-null  object 
 2   review_count   10413 non-null  int64  
 3   categories     10413 non-null  object 
 4   rating         10413 non-null  float64
 5   coordinates    7677 non-null   object 
 6   phone          9216 non-null   float64
 7   display_phone  9216 non-null   object 
 8   price          6951 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 813.5+ KB
None


In [24]:
# Split Coordinates column into lat, long
master_df_sub[['coord_lat', 'coord_long']] = master_df_sub['coordinates'].str.split(pat=', ', expand=True)
master_df_sub = master_df_sub.drop('coordinates', axis=1)

# Rename columns
master_df_sub.columns = ['id', 'name', 'review_count', 'category', 'rating', 'phone', 'display_phone', 'price', 'coord_lat', 'coord_long']
master_df_sub

,id,name,review_count,category,rating,phone,display_phone,price,coord_lat,coord_long
0,epvSBt9LldIgyc08f8nIAw,La Bicyclette Bakery,3,bakeries,4.5,NaN,NaN,NaN,40.68358704921419,-73.99509019999999
2,-k_5NsYnKCHGTdBaqegkrw,Le Fournil,118,bakeries,4.5,NaN,NaN,$$,40.727947,-73.988489
4,0kW0112jMERVjpRzWT4F7Q,La Bicyclette Bakery,84,bakeries,5.0,1.347916e+10,(347) 916-1417,NaN,40.714400045936806,-73.95911000669003
6,XSwagBljEsxG2i1rq4GEAQ,Almondine Bakery,404,bakeries,4.0,1.718798e+10,(718) 797-5026,$$,40.7033174,-73.9912506
8,fPQ2eE9lm8tc87O5-GLjyA,Mille-Feuille Bakery,441,coffee,4.5,1.212363e+10,(212) 362-6261,$$,40.782425,-73.9811603
...,...,...,...,...,...,...,...,...,...,...
29808,dt5ze7syLkALVI3DQfHp_A,Loqma Cafe,20,cafes,3.5,NaN,NaN,$$,40.585861,-73.953604
29810,dt5ze7syLkALVI3DQfHp_A,Loqma Cafe,20,seafood,3.5,NaN,NaN,$$,40.585861,-73.953604
29812,raqwXUBlnc0e8c7CYmxEJQ,Scarpetta,2264,italian,4.0,1.212691e+10,(212) 691-0555,$$$,40.744509,-73.9856
29814,raqwXUBlnc0e8c7CYmxEJQ,Scarpetta,2264,bars,4.0,1.212691e+10,(212) 691-0555,$$$,40.744509,-73.9856


In [25]:
# Identify redundant categories, drop categories not of interest
    # Ice Cream
master_df_sub['category'] = master_df_sub['category'].str.replace('icecream&frozenyogurt', 'icecream')
    # Cake shops
master_df_sub['category'] = master_df_sub['category'].str.replace('patisserie/cakeshop', 'cakeshop')
master_df_sub['category'] = master_df_sub['category'].str.replace('customcakes', 'cakeshop')
    # Gelato
master_df_sub['category'] = master_df_sub['category'].str.replace('gelato', 'icecream')
    # Venues
master_df_sub['category'] = master_df_sub['category'].str.replace('venues&eventspaces', 'venues')
    # Coffee and teas
master_df_sub['category'] = master_df_sub['category'].str.replace('coffee&tea', 'coffee')
    # Coffee and teas
master_df_sub['category'] = master_df_sub['category'].str.replace('juicebars&smoothies', 'juicebars')
    # Coffee and teas
master_df_sub['category'] = master_df_sub['category'].str.replace('chocolatiers&shops', 'chocolate')

# Drop Categories
master_df_sub.drop_duplicates(subset=['id', 'category'], inplace=True, keep='first')

In [26]:
master_df_sub.category.value_counts()

category
nan                 2736
bakeries            1145
coffee               861
desserts             656
breakfast&brunch     295
                    ... 
jewelry                1
petadoption            1
artgalleries           1
bangladeshi            1
beveragestore          1
Name: count, Length: 260, dtype: int64

In [27]:
master_df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9108 entries, 0 to 29816
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             9108 non-null   object 
 1   name           9108 non-null   object 
 2   review_count   9108 non-null   int64  
 3   category       9108 non-null   object 
 4   rating         9108 non-null   float64
 5   phone          8088 non-null   float64
 6   display_phone  8088 non-null   object 
 7   price          6096 non-null   object 
 8   coord_lat      6372 non-null   object 
 9   coord_long     6372 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 782.7+ KB


In [28]:
# Export to csv
# master_df_sub.to_csv('Resources/nyc_bakeries_aggregated.csv')

In [29]:
master_df_sub

,id,name,review_count,category,rating,phone,display_phone,price,coord_lat,coord_long
0,epvSBt9LldIgyc08f8nIAw,La Bicyclette Bakery,3,bakeries,4.5,NaN,NaN,NaN,40.68358704921419,-73.99509019999999
2,-k_5NsYnKCHGTdBaqegkrw,Le Fournil,118,bakeries,4.5,NaN,NaN,$$,40.727947,-73.988489
4,0kW0112jMERVjpRzWT4F7Q,La Bicyclette Bakery,84,bakeries,5.0,1.347916e+10,(347) 916-1417,NaN,40.714400045936806,-73.95911000669003
6,XSwagBljEsxG2i1rq4GEAQ,Almondine Bakery,404,bakeries,4.0,1.718798e+10,(718) 797-5026,$$,40.7033174,-73.9912506
8,fPQ2eE9lm8tc87O5-GLjyA,Mille-Feuille Bakery,441,coffee,4.5,1.212363e+10,(212) 362-6261,$$,40.782425,-73.9811603
...,...,...,...,...,...,...,...,...,...,...
29808,dt5ze7syLkALVI3DQfHp_A,Loqma Cafe,20,cafes,3.5,NaN,NaN,$$,40.585861,-73.953604
29810,dt5ze7syLkALVI3DQfHp_A,Loqma Cafe,20,seafood,3.5,NaN,NaN,$$,40.585861,-73.953604
29812,raqwXUBlnc0e8c7CYmxEJQ,Scarpetta,2264,italian,4.0,1.212691e+10,(212) 691-0555,$$$,40.744509,-73.9856
29814,raqwXUBlnc0e8c7CYmxEJQ,Scarpetta,2264,bars,4.0,1.212691e+10,(212) 691-0555,$$$,40.744509,-73.9856
